In [1]:
!pip install -U openai-whisper
!apt install ffmpeg
!pip install dropbox

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:4.2.7-0ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import whisper
import torch
from whisper.utils import get_writer

In [3]:
from google.colab import drive
import time
import glob
from pathlib import Path
import glob
import requests
import json
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:

Model_Type = "large"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(DEVICE)
#load Whipser model
model = whisper.load_model(Model_Type, device=DEVICE)


cuda


In [5]:
!nvidia-smi

Tue Apr 25 17:08:17 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P0    27W /  70W |   9973MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

ref: https://hackmd.io/lAP-vUyeQ-KCQVfbhNJrPA
```js
function ClickConnect(){
    console.log("Working"); 
    document.querySelector("colab-toolbar-button#connect").click() 
}
setInterval(ClickConnect,60000)
```


## asr

In [7]:
def asr(input_file, model, output_directory='file_path_on_google_dirve'):
    Data_File = input_file
    TextFileName = Path(Data_File).stem
    file_name = f"{TextFileName}"
    #output_directory = "./aa"
    t = time.time()
    result = model.transcribe(Data_File)
    print(time.time()-t)

    # save TXT
    txt_writer = get_writer("txt", output_directory)
    txt_writer(result, file_name)


    # save SRT
    srt_writer = get_writer("srt", output_directory)
    srt_writer(result, file_name)
    
    return result


In [15]:
def translate_text(text):
    start_t = time.time()
    wait_time = 30
    error = 0
    while True:
        api_key = "<ms_token>"
        url = "https://ifopenairesourcedev.openai.azure.com/openai/deployments/gpt-4-32k/chat/completions?api-version=2023-03-15-preview"


        headers = {"Content-Type": "application/json", "api-key": api_key}
        parameters = {
           "messages": [
                {"role": "system", "content": "你是逐字稿處理器，將逐字稿重組成會議總結，並加上標題，子標題與內文，特別是有數據的地方、有比較的內容都用表格呈現。有其他非會議主要內容，也請另外列點說明，全文統一用markdown輸出，最後無法與前面整合的保留原逐字稿內文用 {{未處理內文}} 輸出"},
                {"role": "user", "content": text},
          ],
         "max_tokens": 5000,
          "temperature": 0.9,
          "frequency_penalty": 0,
          "presence_penalty": 0,
          "top_p": 1
        }
        response = requests.post(url, headers=headers, json=parameters)
        if response.status_code != 200:
            error += 1
            time.sleep(wait_time)
            print("ERROR" +str(response.status_code))
            wait_time = wait_time*2            
        else:
            ans = response.json()['choices'][0]['message']['content']            
            break
    print(f'output: {time.time()-start_t}')
    return ans


In [9]:
def do(q):
    text = translate_text(q)
    content_start = text.find("{{") + 2
    content_end = text.find("}}")
    
    content = text[content_start:content_end].strip()
    remaining_text = text[:content_start - 2] + text[content_end + 2:]
    
    return content, remaining_text


## Export to hackmd

In [10]:
def gen_hackmd(file_name,total_ans,ss):
    token = "<hackmd_token>"  # 替換為您的Bearer Token
    url = "https://api.hackmd.io/v1/notes"  # 替換為您要訪問的API URL

    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json",
    }

    info = "\n\n".join(total_ans)
    doc = '\n'.join(ss)
    data = {
        "content": f"# {Path(file_name).stem}\n###### tags: `esun`,`gpt`\n\n{info}\n\n## 逐字稿\n {doc}",
        "readPermission": "owner",
        "writePermission": "owner",
        "commentPermission": "owner"
    }

    response = requests.post(url, headers=headers, data=json.dumps(data))
    response_data = response.json()
    #print(response_data['id'])
    response_data
    doc_id = response_data['id']
    print(doc_id)

## main

In [ ]:
files = glob.glob('path_of_audio_files')
len(files),files

In [ ]:
output_directory='path_of_output'
for file_name in files:
    print(Path(file_name).stem)
    result = asr(file_name, model,output_directory)
    ff = f'{output_directory}/{Path(file_name).stem}.txt'
    print(ff)
    with open(ff,'r') as f:
        lines = f.readlines()
        ss = [line[:-1] for line in lines]
    parts = []
    counter = []
    total_ans = []
    for s in ss:
        counter.append(s)
        q = "\n".join(counter)
        if len(q) >= 13000:
            rest,ans = do(q)
            counter = [rest]
            total_ans.append(ans)
    if counter:
        q = "\n".join(counter)
        rest,ans = do(q)
        total_ans.append(ans)
        total_ans.append(rest)
    gen_hackmd(ff,total_ans,ss)